In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from torch import nn

In [111]:
class MyNeuralNetwork(nn.Module):
    def __init__(self, l1 = 100) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, l1),
            nn.LeakyReLU(),
            nn.Linear(l1, 7)
        )
    
    def forward(self, inputs):
        return self.net(inputs)

In [86]:
class MultiEmoVA(Dataset):
    def __init__(self, data_path) -> None:
        super().__init__()
        data=pd.read_csv(data_path)
        if 'emotion' in data:
            self.inputs = torch.tensor(data.drop("emotion", axis=1).to_numpy(np.float32))
            self.index2label = [label for label in data["emotion"].unique()]
            label2index = {label: i for i, label in enumerate(self.index2label)}
            self.labels = torch.tensor(data["emotion"].apply(lambda x: label2index[x]))
        else:
            self.inputs = torch.tensor(data.to_numpy(np.float32))
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        if hasattr(self, 'labels'):
            return self.inputs[index], self.labels[index]
        else:
            return self.inputs[index]
    
    @staticmethod
    def label_dict():
        data=pd.read_csv("../dataset/dataset.csv")
        return [label for label in data["emotion"].unique()]
        

In [112]:
data = MultiEmoVA("../dataset/dataset.csv")
unlabeled = MultiEmoVA("../dataset/test_to_submit.csv")
#print(data[0])
train, val, test = random_split(data, [0.7, 0.2, 0.1])

model = MyNeuralNetwork()


train_loader = DataLoader(train)
val_loader = DataLoader(val)
test_loader = DataLoader(test)

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.01)

n_epochs =20

for epoch in range(n_epochs):
    for inputs, labels in train_loader:
        output = model(inputs)
        loss = loss_fn(output, labels)
        loss.backward()
        optim.step()
        optim.zero_grad()

    correct = 0
    for inputs, labels in val_loader:
        output = model(inputs)
        correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
        print(correct)
    print(f"Validation accuracy is: {correct/len(val)}")


correct = 0
for inputs, labels, in test_loader:
    output = model(inputs)
    correct += (output.softmax(dim=1).argmax(dim=1) == labels).sum()
print(f"Test accuracy is: {correct/len(test)}")
    
unlabeled_loader = DataLoader(unlabeled)

out = pd.DataFrame(list(map(lambda x: MultiEmoVA.label_dict()[model(x).softmax(dim=1).argmax(dim=1)], unlabeled_loader)))
out.to_csv('../processed/outputs', index = False, header = False)


tensor(1)
tensor(2)
tensor(2)
tensor(3)
tensor(4)
tensor(5)
tensor(5)
tensor(5)
tensor(6)
tensor(6)
tensor(7)
tensor(8)
tensor(8)
tensor(8)
tensor(9)
tensor(9)
tensor(9)
tensor(9)
tensor(10)
tensor(10)
tensor(11)
tensor(11)
tensor(11)
tensor(11)
tensor(11)
tensor(12)
tensor(12)
tensor(13)
tensor(14)
tensor(14)
tensor(15)
tensor(16)
tensor(16)
tensor(17)
tensor(17)
tensor(17)
tensor(17)
tensor(18)
tensor(18)
tensor(19)
tensor(19)
tensor(19)
tensor(19)
tensor(20)
tensor(20)
tensor(21)
tensor(22)
tensor(22)
tensor(22)
tensor(22)
tensor(23)
tensor(24)
tensor(24)
tensor(25)
tensor(25)
tensor(25)
tensor(25)
tensor(25)
tensor(26)
tensor(26)
tensor(27)
tensor(27)
tensor(28)
tensor(29)
tensor(29)
tensor(30)
tensor(30)
tensor(30)
tensor(31)
tensor(31)
tensor(31)
tensor(31)
tensor(32)
tensor(32)
tensor(33)
tensor(34)
tensor(35)
tensor(35)
tensor(36)
tensor(36)
tensor(37)
tensor(38)
tensor(39)
tensor(40)
tensor(40)
tensor(41)
tensor(42)
tensor(42)
tensor(42)
tensor(43)
tensor(44)
tensor(45)
tensor